In [51]:
import skforecast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from dateutil.relativedelta import relativedelta
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

In [86]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
test = list(df['PLAYER'].unique()[:10])
data = df[df['PLAYER'].isin(test)]
test

['Christian McCaffrey',
 'Tyreek Hill',
 'CeeDee Lamb',
 'Josh Allen',
 'Raheem Mostert',
 'Jalen Hurts',
 'Keenan Allen',
 'Mike Evans',
 'A.J. Brown',
 'Travis Etienne']

In [93]:
df = pd.read_csv("database/DIM_PLAYER_GAMES.csv")
results = []
for player in test:
    data = df.loc[df["PLAYER"] == player]
    data["FORECAST_DATE"] = pd.date_range("2018-01-01", periods=len(data), freq="MS")
    data["FORECAST_DATE"] = pd.to_datetime(data["FORECAST_DATE"], format="%Y-%m-%d")
    data = data.set_index("FORECAST_DATE")
    data = data.asfreq("MS")
    data = data["PPR"]
    end_train = max(data.index) - relativedelta(months=16)
    forecaster = ForecasterAutoreg(regressor=RandomForestRegressor(), lags=7)

    forecaster.fit(y=data.loc[: max(data.index) - relativedelta(months=16)])
    predictions = forecaster.predict(steps=17)
    error_mape = mean_absolute_percentage_error(
        y_true=data.loc[end_train:], y_pred=predictions
    )
    results.append(
        [
            player,
            np.sum(data.loc[end_train:]),
            np.sum(predictions),
            abs(np.sum(data.loc[end_train:]) - np.sum(predictions))
            / np.sum(data.loc[end_train:]),
            error_mape,
            len(data.loc[end_train:]),
            forecaster.regressor,
        ]
    )

C:\Users\012356.CORPORATE\AppData\Local\Temp\ipykernel_27600\1510427478.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["FORECAST_DATE"] = pd.date_range("2018-01-01", periods=len(data), freq="MS")
C:\Users\012356.CORPORATE\AppData\Local\Temp\ipykernel_27600\1510427478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["FORECAST_DATE"] = pd.to_datetime(data["FORECAST_DATE"], format="%Y-%m-%d")
C:\Users\012356.CORPORATE\AppData\Local\Temp\ipykernel_27600\1510427478.py:5: SettingWithCopyWarning:

In [95]:
df_results = pd.DataFrame(results, columns=['PLAYER', 'ACTUAL SEASON TOTAL', 'PREDICTED', 'SEASON MAPE',
                                             'GAME MAPE', 'GAMES PREDICTED', 'REGRESSOR'])
df_results

,PLAYER,ACTUAL SEASON TOTAL,PREDICTED,SEASON MAPE,GAME MAPE,GAMES PREDICTED,REGRESSOR
0,Christian McCaffrey,416.20,430.8576,0.035218,0.281795,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
1,Tyreek Hill,388.90,316.2280,0.186866,0.551241,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
2,CeeDee Lamb,363.80,238.5250,0.344351,0.372531,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
3,Josh Allen,436.86,390.4794,0.106168,0.227201,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
4,Raheem Mostert,276.50,156.4070,0.434333,0.431043,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
5,Jalen Hurts,452.56,369.5252,0.183478,0.295156,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
6,Keenan Allen,369.26,288.1870,0.219555,0.359168,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
7,Mike Evans,308.40,252.1360,0.182438,0.514224,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
8,A.J. Brown,333.00,274.2310,0.176483,0.767409,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
9,Travis Etienne,266.20,54.4000,0.795642,0.698151,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
